# Real esate advertisement analysis 

In [227]:
import hashlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

### Random seed based on NEPTUN code

In [228]:
neptun = "JPWF8N"
seed = int(hashlib.sha256(neptun.encode("utf-8")).hexdigest(), 16) % 10**8
print(f"Random seed based on NEPTUN code: {seed}")

Random seed based on NEPTUN code: 75628879


## Data prep

In [229]:
df = pd.read_csv("../data/ingatlan.csv")

In [230]:
cols_shuffled = np.random.RandomState(seed).permutation(df.columns)
df = df[cols_shuffled]

In [231]:
df.describe()

,ad_view_cnt,nr,balcony_area,price_created_at,postcode,active_days,district,property_area,small_room_cnt,room_cnt
count,183564.000000,183565.000000,175966.000000,183565.000000,115475.000000,183565.000000,176009.000000,183565.000000,182981.000000,183565.000000
mean,262.264082,196659.747942,7.748792,20.564001,1103.395895,44.071593,9.730434,48.443984,0.547177,1.476393
std,556.838684,113179.241269,2360.665258,171.496565,50.789818,47.969011,4.782807,12.776044,0.748810,0.972892
min,0.000000,7.000000,0.000000,0.000000,1011.000000,1.000000,1.000000,0.000000,-1.000000,0.000000
25%,41.000000,98117.000000,0.000000,13.200000,1063.000000,11.000000,6.000000,40.000000,0.000000,1.000000
50%,102.000000,196774.000000,0.000000,16.900000,1101.000000,28.000000,10.000000,50.000000,0.000000,1.000000
75%,265.000000,294824.000000,3.000000,23.900000,1141.000000,60.000000,13.000000,60.000000,1.000000,2.000000
max,40248.000000,394178.000000,990257.000000,41796.000000,1239.000000,544.000000,20.000000,70.000000,56.000000,215.000000


In [232]:
df.isna().sum() # Show number of missing values

orientation                 72142
ad_view_cnt                     1
view_type                   83103
heating_type                26686
nr                              0
property_condition_type         0
balcony_area                 7599
price_created_at                0
property_subtype             3640
elevator_type               33502
postcode                    68090
county                          0
active_days                     0
created_at                      0
building_floor_count        97789
garden_access              142886
district                     7556
property_area                   0
city                         1304
property_floor               9066
property_type                   0
small_room_cnt                584
room_cnt                        0
dtype: int64

In [233]:
df = df.drop(columns=["ad_view_cnt", "active_days", "nr"])

### Target variable

In [234]:
df["price_per_m2"] = df["price_created_at"] / df["property_area"]
df["price_per_m2"].describe()

/home/laszlo-abrok/Projects/machine-learning-case-studies/.venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


count    1.835650e+05
mean              inf
std               NaN
min      0.000000e+00
25%      2.880000e-01
50%      3.760000e-01
75%      4.923077e-01
max               inf
Name: price_per_m2, dtype: float64

In [235]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183565 entries, 0 to 183564
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   orientation              111423 non-null  object 
 1   view_type                100462 non-null  object 
 2   heating_type             156879 non-null  object 
 3   property_condition_type  183565 non-null  object 
 4   balcony_area             175966 non-null  float64
 5   price_created_at         183565 non-null  float64
 6   property_subtype         179925 non-null  object 
 7   elevator_type            150063 non-null  object 
 8   postcode                 115475 non-null  float64
 9   county                   183565 non-null  object 
 10  created_at               183565 non-null  object 
 11  building_floor_count     85776 non-null   object 
 12  garden_access            40679 non-null   object 
 13  district                 176009 non-null  float64
 14  prop

### Data modification

In [236]:
def remove_outliers_iqr(df, column, factor=1.5):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - factor * iqr
    upper = q3 + factor * iqr
    return df[(df[column] >= lower) & (df[column] <= upper)]

In [237]:
def floor_parser(val):
    if pd.isna(val):
        return np.nan

    val = str(val).strip().lower()

    if "basement" in val:
        return -1
    elif "ground" in val:
        return 0
    elif "mezzanine" in val:
        return 0.5
    elif "plus" in val:
        nums = [int(s) for s in val.split() if s.isdigit()]
        return nums[0] + 1 if nums else 11
    else:
        try:
            return int(val)
        except ValueError:
            return np.nan

In [238]:
def building_floor_count_parser(value):
    # distinct values are: 1-10, more than 10, nan
    if pd.isna(value):
        return np.nan

    value = str(value).strip().lower()

    if "more" in value and "10" in value:
        return 11
    else:
        try:
            return int(value)
        except ValueError:
            return np.nan

In [239]:
# df['building_floor_count'].unique()
# values = ['10', np.nan, '4', '3', '2', '5', '7', '9', '6', '1', 'more than 10', '8']
# print([parse_building_floor_count(v) for v in values])


In [240]:
#df['property_floor'].unique()

# floors = ['3', 'ground floor', '4', 'basement', '7', '2', '1', np.nan, 'mezzanine floor', '8', '6', '10', '5', '9', '10 plus']
#print([floor_parser(x) for x in floors])


In [241]:
df['property_floor'] = df['property_floor'].apply(floor_parser)
df['building_floor_count'] = df['building_floor_count'].apply(building_floor_count_parser)


In [242]:
df.describe()

/home/laszlo-abrok/Projects/machine-learning-case-studies/.venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,balcony_area,price_created_at,postcode,building_floor_count,district,property_area,property_floor,small_room_cnt,room_cnt,price_per_m2
count,175966.000000,183565.000000,115475.000000,85776.000000,176009.000000,183565.000000,174499.000000,182981.000000,183565.000000,1.835650e+05
mean,7.748792,20.564001,1103.395895,5.378544,9.730434,48.443984,2.636161,0.547177,1.476393,inf
std,2360.665258,171.496565,50.789818,2.995146,4.782807,12.776044,2.579034,0.748810,0.972892,NaN
min,0.000000,0.000000,1011.000000,1.000000,1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000e+00
25%,0.000000,13.200000,1063.000000,3.000000,6.000000,40.000000,1.000000,0.000000,1.000000,2.880000e-01
50%,0.000000,16.900000,1101.000000,4.000000,10.000000,50.000000,2.000000,0.000000,1.000000,3.760000e-01
75%,3.000000,23.900000,1141.000000,9.000000,13.000000,60.000000,4.000000,1.000000,2.000000,4.923077e-01
max,990257.000000,41796.000000,1239.000000,11.000000,20.000000,70.000000,11.000000,56.000000,215.000000,inf


In [243]:
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
df['month'] = df['created_at'].dt.month
df['year'] = df['created_at'].dt.year
df['day'] = df['created_at'].dt.day
df = df.drop(columns=['created_at'])

In [244]:
df.describe()

/home/laszlo-abrok/Projects/machine-learning-case-studies/.venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,balcony_area,price_created_at,postcode,building_floor_count,district,property_area,property_floor,small_room_cnt,room_cnt,price_per_m2,month,year,day
count,175966.000000,183565.000000,115475.000000,85776.000000,176009.000000,183565.000000,174499.000000,182981.000000,183565.000000,1.835650e+05,183565.000000,183565.000000,183565.000000
mean,7.748792,20.564001,1103.395895,5.378544,9.730434,48.443984,2.636161,0.547177,1.476393,inf,5.875515,2015.385662,15.328532
std,2360.665258,171.496565,50.789818,2.995146,4.782807,12.776044,2.579034,0.748810,0.972892,NaN,3.119566,0.486753,8.708227
min,0.000000,0.000000,1011.000000,1.000000,1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000e+00,1.000000,2015.000000,1.000000
25%,0.000000,13.200000,1063.000000,3.000000,6.000000,40.000000,1.000000,0.000000,1.000000,2.880000e-01,3.000000,2015.000000,8.000000
50%,0.000000,16.900000,1101.000000,4.000000,10.000000,50.000000,2.000000,0.000000,1.000000,3.760000e-01,6.000000,2015.000000,15.000000
75%,3.000000,23.900000,1141.000000,9.000000,13.000000,60.000000,4.000000,1.000000,2.000000,4.923077e-01,8.000000,2016.000000,23.000000
max,990257.000000,41796.000000,1239.000000,11.000000,20.000000,70.000000,11.000000,56.000000,215.000000,inf,12.000000,2016.000000,31.000000


In [245]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
print("Numerical columns:", num_cols)
print("Categorical columns:", cat_cols)
print(df[num_cols].isna().sum())
print(df[cat_cols].isna().sum())

Numerical columns: ['balcony_area', 'price_created_at', 'postcode', 'building_floor_count', 'district', 'property_area', 'property_floor', 'small_room_cnt', 'room_cnt', 'price_per_m2', 'month', 'year', 'day']
Categorical columns: ['orientation', 'view_type', 'heating_type', 'property_condition_type', 'property_subtype', 'elevator_type', 'county', 'garden_access', 'city', 'property_type']
balcony_area             7599
price_created_at            0
postcode                68090
building_floor_count    97789
district                 7556
property_area               0
property_floor           9066
small_room_cnt            584
room_cnt                    0
price_per_m2                0
month                       0
year                        0
day                         0
dtype: int64
orientation                 72142
view_type                   83103
heating_type                26686
property_condition_type         0
property_subtype             3640
elevator_type               33502
co

In [246]:
df[num_cols] = df[num_cols].fillna(df[num_cols].median())
df[cat_cols] = df[cat_cols].fillna("missing")
print(df.isna().sum())
df.describe()

orientation                0
view_type                  0
heating_type               0
property_condition_type    0
balcony_area               0
price_created_at           0
property_subtype           0
elevator_type              0
postcode                   0
county                     0
building_floor_count       0
garden_access              0
district                   0
property_area              0
city                       0
property_floor             0
property_type              0
small_room_cnt             0
room_cnt                   0
price_per_m2               0
month                      0
year                       0
day                        0
dtype: int64


/home/laszlo-abrok/Projects/machine-learning-case-studies/.venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,balcony_area,price_created_at,postcode,building_floor_count,district,property_area,property_floor,small_room_cnt,room_cnt,price_per_m2,month,year,day
count,183565.000000,183565.000000,183565.000000,183565.000000,183565.000000,183565.000000,183565.000000,183565.000000,183565.000000,1.835650e+05,183565.000000,183565.000000,183565.000000
mean,7.428017,20.564001,1102.507183,4.644164,9.741530,48.443984,2.604742,0.545436,1.476393,inf,5.875515,2015.385662,15.328532
std,2311.287103,171.496565,40.299919,2.159849,4.683642,12.776044,2.518315,0.748253,0.972892,NaN,3.119566,0.486753,8.708227
min,0.000000,0.000000,1011.000000,1.000000,1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000e+00,1.000000,2015.000000,1.000000
25%,0.000000,13.200000,1084.000000,4.000000,6.000000,40.000000,1.000000,0.000000,1.000000,2.880000e-01,3.000000,2015.000000,8.000000
50%,0.000000,16.900000,1101.000000,4.000000,10.000000,50.000000,2.000000,0.000000,1.000000,3.760000e-01,6.000000,2015.000000,15.000000
75%,3.000000,23.900000,1118.000000,4.000000,13.000000,60.000000,4.000000,1.000000,2.000000,4.923077e-01,8.000000,2016.000000,23.000000
max,990257.000000,41796.000000,1239.000000,11.000000,20.000000,70.000000,11.000000,56.000000,215.000000,inf,12.000000,2016.000000,31.000000


In [247]:
# export dataframe for verification
# df.to_csv("../data/ingatlan_cleaned.csv", index=False)

In [248]:
# print outliers in property_area and balcony_area

print(f"number of property area: {len(df['property_area'])}")

property_area_outliers = df[~df['property_area'].between(df['property_area'].quantile(0.01), df['property_area'].quantile(0.99))]
balcony_area_outliers = df[~df['balcony_area'].between(df['balcony_area'].quantile(0.01), df['balcony_area'].quantile(0.99))]
price_per_m2_outliers = df[~df['price_per_m2'].between(df['price_per_m2'].quantile(0.01), df['price_per_m2'].quantile(0.99))]

print(f"number of property area outliers: {len(property_area_outliers)}")
print(f"number of balcony area outliers: {len(balcony_area_outliers)}")
print(f"number of price per m2 outliers: {len(price_per_m2_outliers)}")


number of property area: 183565
number of property area outliers: 708
number of balcony area outliers: 1597
number of price per m2 outliers: 3669


### Removing outliers

In [249]:
df = remove_outliers_iqr(df, 'property_area', factor=3)
df = remove_outliers_iqr(df, 'balcony_area', factor=3)
df = remove_outliers_iqr(df, 'price_per_m2', factor=3)

print(f"number of property area after outlier removal: {len(df)}")


number of property area after outlier removal: 178459


### Drop columns deemed unnecessary

In [250]:
# show unique values for each categorical column
for col in cat_cols:
    print(f"Unique values in column '{col}': {df[col].unique()}")


Unique values in column 'orientation': ['east' 'missing' 'west' 'south-east' 'south-west' 'north-east'
 'north-west' 'south' 'north']
Unique values in column 'view_type': ['garden view' 'missing' 'street view' 'courtyard view' 'panoramic']
Unique values in column 'heating_type': ['missing' 'gas furnace, circulating hot water' 'konvection gas burner'
 'district heating' 'central heating with own meter' 'tile stove (gas)'
 'central heating' 'electric' 'other' 'fan-coil' 'gas furnace']
Unique values in column 'property_condition_type': ['good' 'novel' 'medium' 'renewed' 'new_construction' 'to_be_renovated'
 'can_move_in' 'missing_info' 'under_construction']
Unique values in column 'property_subtype': ['prefabricated panel flat (for sale)' 'brick flat (for sale)' 'missing'
 'prefabricated panel flat (for rent)' 'terraced house']
Unique values in column 'elevator_type': ['yes' 'missing' 'none']
Unique values in column 'county': ['Budapest']
Unique values in column 'garden_access': ['missing

In [251]:
df.drop(columns=['county','property_type','city'], inplace=True)

In [252]:
df.describe()

,balcony_area,price_created_at,postcode,building_floor_count,district,property_area,property_floor,small_room_cnt,room_cnt,price_per_m2,month,year,day
count,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000
mean,1.457113,18.995415,1102.499302,4.653125,9.741033,48.352591,2.609501,0.542063,1.470517,0.400180,5.882163,2015.384828,15.324607
std,2.503852,8.392615,40.298179,2.170813,4.680053,12.703993,2.522448,0.725174,0.895132,0.151379,3.117738,0.486556,8.710277
min,0.000000,0.000000,1011.000000,1.000000,1.000000,5.000000,-1.000000,-1.000000,0.000000,0.000000,1.000000,2015.000000,1.000000
25%,0.000000,13.000000,1083.000000,4.000000,6.000000,40.000000,1.000000,0.000000,1.000000,0.285714,3.000000,2015.000000,8.000000
50%,0.000000,16.800000,1101.000000,4.000000,10.000000,50.000000,2.000000,0.000000,1.000000,0.371429,6.000000,2015.000000,15.000000
75%,3.000000,23.400000,1118.000000,4.000000,13.000000,60.000000,4.000000,1.000000,2.000000,0.483333,8.000000,2016.000000,23.000000
max,12.000000,74.400000,1239.000000,11.000000,20.000000,70.000000,11.000000,47.000000,215.000000,1.085714,12.000000,2016.000000,31.000000


In [253]:
# save df in csv format
df.to_csv("../data/ingatlan_cleaned.csv", index=False)

### Get target

In [254]:
# y = df['price_per_m2'].copy()
# df = df.drop(columns=['price_per_m2'])

### Encoding

In [255]:
# y_log = np.log1p(y)

In [256]:
def apply_target_encoding(df, target, cols):
    df_encoded = df.copy()
    for col in cols:
        means = df_encoded.groupby(col)[target].mean()
        df_encoded[col + '_te'] = df_encoded[col].map(means)
        df_encoded = df_encoded.drop(columns=[col])
    return df_encoded

#### Target encoding

In [257]:
target_cols = ['postcode', 'district']

In [258]:
df_encoded = df.copy()
df_encoded['price_per_m2'] = np.log1p(df_encoded['price_per_m2'])  # ideiglenesen visszatesszük a targetet a target encodinghoz
df_encoded = apply_target_encoding(df_encoded, 'price_per_m2', target_cols)
#df_encoded = df_encoded.drop(columns=['price_per_m2'])


#### Ordinal encoding

In [259]:
ordinal_cols = [['to_be_renovated',
    'missing_info',
    'medium',
    'renewed',
    'good',
    'can_move_in',
    'under_construction',
    'novel',
    'new_construction']]

In [260]:
ord_enc = OrdinalEncoder(categories=ordinal_cols)
df_encoded['property_condition_type_encoded'] = ord_enc.fit_transform(
    df_encoded[['property_condition_type']]
)
df_encoded = df_encoded.drop(columns=['property_condition_type'])

#### One-Hot encoding  

In [261]:
onehot_cols = [
    'property_subtype', 'property_subtype', 'view_type',
    'orientation', 'heating_type', 'elevator_type'
]

In [262]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded = ohe.fit_transform(df_encoded[onehot_cols])
encoded_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(onehot_cols))
df_encoded = pd.concat([df_encoded.drop(columns=onehot_cols), encoded_df], axis=1)

In [263]:
df_encoded.describe()

,balcony_area,price_created_at,building_floor_count,property_area,property_floor,small_room_cnt,room_cnt,price_per_m2,month,year,...,heating_type_fan-coil,heating_type_gas furnace,"heating_type_gas furnace, circulating hot water",heating_type_konvection gas burner,heating_type_missing,heating_type_other,heating_type_tile stove (gas),elevator_type_missing,elevator_type_none,elevator_type_yes
count,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,...,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.000000,178459.00000
mean,1.457113,18.995415,4.653125,48.352591,2.609501,0.542063,1.470517,0.331079,5.882163,2015.384828,...,0.001306,0.000118,0.184418,0.275822,0.147485,0.011196,0.016609,0.182378,0.375672,0.44195
std,2.503852,8.392615,2.170813,12.703993,2.522448,0.725174,0.895132,0.103741,3.117738,0.486556,...,0.036110,0.010847,0.387826,0.446929,0.354590,0.105217,0.127801,0.386157,0.484297,0.49662
min,0.000000,0.000000,1.000000,5.000000,-1.000000,-1.000000,0.000000,0.000000,1.000000,2015.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,0.000000,13.000000,4.000000,40.000000,1.000000,0.000000,1.000000,0.251314,3.000000,2015.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,0.000000,16.800000,4.000000,50.000000,2.000000,0.000000,1.000000,0.315853,6.000000,2015.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,3.000000,23.400000,4.000000,60.000000,4.000000,1.000000,2.000000,0.394292,8.000000,2016.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.00000
max,12.000000,74.400000,11.000000,70.000000,11.000000,47.000000,215.000000,0.735111,12.000000,2016.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


### Numeric scaling

In [264]:
numeric_cols = [
    'property_area', 'balcony_area', 'property_floor',
    'building_floor_count', 'room_cnt', 'small_room_cnt',
    'year', 'month', 'day'
]

In [265]:
scaler = StandardScaler()
df_encoded[numeric_cols] = scaler.fit_transform(df_encoded[numeric_cols])

In [266]:
# get count of missing values in each column
print(df_encoded.isna().sum())

balcony_area                                            4971
price_created_at                                        4971
building_floor_count                                    4971
garden_access                                           4971
property_area                                           4971
property_floor                                          4971
small_room_cnt                                          4971
room_cnt                                                4971
price_per_m2                                            4971
month                                                   4971
year                                                    4971
day                                                     4971
postcode_te                                             4971
district_te                                             4971
property_condition_type_encoded                         4971
property_subtype_brick flat (for sale)                  4971
property_subtype_missing

In [267]:
df_encoded['garden_access_binary'] = df['garden_access'].map({'yes': 1, 'none': 0}).fillna(-1)
df_encoded.drop(columns=['garden_access'], inplace=True)

In [268]:
df_encoded = df_encoded.fillna(df_encoded.median(numeric_only=True))

In [269]:
df_encoded.drop(columns=['price_created_at'], inplace=True)

In [270]:
print(df_encoded.isna().sum())

balcony_area                                            0
building_floor_count                                    0
property_area                                           0
property_floor                                          0
small_room_cnt                                          0
room_cnt                                                0
price_per_m2                                            0
month                                                   0
year                                                    0
day                                                     0
postcode_te                                             0
district_te                                             0
property_condition_type_encoded                         0
property_subtype_brick flat (for sale)                  0
property_subtype_missing                                0
property_subtype_prefabricated panel flat (for rent)    0
property_subtype_prefabricated panel flat (for sale)    0
property_subty

In [271]:
print(df_encoded.dtypes)

balcony_area                                            float64
building_floor_count                                    float64
property_area                                           float64
property_floor                                          float64
small_room_cnt                                          float64
room_cnt                                                float64
price_per_m2                                            float64
month                                                   float64
year                                                    float64
day                                                     float64
postcode_te                                             float64
district_te                                             float64
property_condition_type_encoded                         float64
property_subtype_brick flat (for sale)                  float64
property_subtype_missing                                float64
property_subtype_prefabricated panel fla

In [272]:
# save encoded dataframe
df_encoded.to_csv("../data/ingatlan_encoded.csv", index=False)

In [273]:
X = df_encoded.copy()
X.drop(columns=['price_per_m2'], inplace=True)
y = df_encoded['price_per_m2'].copy()
print(X.shape, y.shape)

(183430, 51) (183430,)
